# Django ORM CookBook

## 정보를 조회하고 필요한 항목을 선별하는 방법

### 1. 장고 ORM이 실행하는 실제 SQL 질의문을 확인할 수 있나요?

In [1]:
from events.models import Event

In [2]:
queryset = Event.objects.all()
str(queryset.query)

'SELECT "events_event"."id", "events_event"."epic_id", "events_event"."details", "events_event"."years_ago" FROM "events_event"'

In [3]:
queryset = Event.objects.filter(years_ago__gt=5)
str(queryset.query)

'SELECT "events_event"."id", "events_event"."epic_id", "events_event"."details", "events_event"."years_ago" FROM "events_event" WHERE "events_event"."years_ago" > 5'

### 2. OR 연산으로 일부 조건을 하나라도 만족하는 항목을 구하려면 어떻게 하나요?

In [4]:
# 데이터 베이스 생성
from django.contrib.auth import get_user_model
get_user_model()

data = [
    ['yash', 'Yash', 'Rastogi', 'yashr@agiliq.com'],
    ['John', 'John', 'Kumar', 'john@gmail.com'],
    ['Ricky', 'Ricky', 'Dayal', 'ricky@gmail.com'],
    ['sharukh', 'Sharukh', 'Misra', 'sharukh@hotmail.com'],
    ['Ritesh', 'Ritesh', 'Deshmukh', 'ritesh@yahoo.com'],
    ['Billy', 'Billy', 'Sharma', 'billy@gmail.com'],
    ['Radha', 'Radha', 'George', 'radha@gmail.com'],
    ['sohan' ,'Sohan', 'Upadhyay', 'sohan@aol.com'],
    ['Raghu', 'Raghu', 'Khan', 'raghu@rediffmail.com'],
    ['rishab', 'Rishabh', 'Deol', 'rishabh@yahoo.com']
]

from django.contrib.auth.models import User

for i in data:
    User.objects.create(
        username=i[0],
        first_name=i[1],
        last_name=i[2],
        email=i[3])

`OR` 연산 // 이름이 'R'로 시작하거나, 성이 'D'로 시작하는 모든 사용자 구하기
- `queryset_1 | queryset_2`
- `filter(Q(<condition_1>)|Q(<condition_2>))`

구하려는 SQL 질의문:  
`SELECT username, first_name, last_name, email FROM auth_user WHERE first_name LIKE 'R%' OR last_name LIKE 'D%';`


In [5]:
'''
- queryset_1 | queryset_2
'''

queryset = User.objects.filter(first_name__startswith='R') | User.objects.filter(last_name__startswith='D')
queryset

<QuerySet [<User: Ricky>, <User: Ritesh>, <User: Radha>, <User: Raghu>, <User: rishab>]>

In [6]:
str(queryset.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE \'\\\' OR "auth_user"."last_name" LIKE D% ESCAPE \'\\\')'

In [7]:
'''
- filter(Q(<condition_1>)|Q(<condition_2>))
'''

from django.db.models import Q
qs = User.objects.filter(Q(first_name__startswith='R')|Q(last_name__startswith='D'))

In [8]:
str(qs.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE \'\\\' OR "auth_user"."last_name" LIKE D% ESCAPE \'\\\')'

### 3. AND 연산으로 여러 조건을 모두 만족하는 항목을 구하려면 어떻게 하나요?

`AND` 연산 // 이름이 'R'로 시작하고 성이 'D'로 시작하는 모든 사용자 구하기

- `filter(<condition_1>, <condition_2>)`
- `queryset_1 & queryset_2`
- `filter(Q(<condition_1>) & Q(<condition_2>))`

구하려는 SQL 질의문:  
`SELECT username, first_name, last_name, email FROM auth_user WHERE first_name LIKE 'R%' AND last_name LIKE 'D%';`

In [9]:
'''
filter(<condition_1>, <condition_2>)
'''
qs1 = User.objects.filter(
    first_name__startswith='R',
    last_name__startswith='D'
    )

In [10]:
'''
queryset_1 & queryset_2
'''
qs2 = User.objects.filter(
    first_name__startswith='R'
    ) & User.objects.filter(
    last_name__startswith='D'
    )

In [11]:
'''
filter(Q(<condition_1>) & Q(<condition_2>))
'''
qs3 = User.objects.filter(
    Q(first_name__startswith='R') &
    Q(last_name__startswith='D')
    )

In [12]:
str(qs1) == str(qs2) == str(qs3)

True

In [13]:
str(qs1.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE \'\\\' AND "auth_user"."last_name" LIKE D% ESCAPE \'\\\')'

### 4. NOT 연산으로 조건을 부정하려면 어떻게 하나요?

id < 5 라는 조건을 만족하지 않는 모든 사용자를 구해 봅시다.  
이를 수행하려면 NOT 연산이 필요합니다.

- `exclude(<condition>)`
- `filter(~Q(<condition>))`

구하려는 SQL 질의문:  
`SELECT id, username, first_name, last_name, email FROM auth_user WHERE NOT id < 5;`

In [14]:
qs1 = User.objects.exclude(id__lt=5)
qs1

<QuerySet [<User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>]>

In [15]:
qs2 = User.objects.filter(~Q(id__lt=5))
qs2

<QuerySet [<User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>]>

### 5. 동일한 모델 또는 서로 다른 모델에서 구한 쿼리셋들을 합할 수 있나요?

SQL에서는 여러 개의 결과 집합을 합할 때 UNION 연산을 이용합니다.  
장고 ORM에서 union 메서드를 이용해 쿼리셋을 합할 수 있습니다.  
합하려는 쿼리셋의 모델이 서로 다른 경우, 각 쿼리셋에 포함된 필드와 데이터 유형이 서로 맞아야 합니다.

In [16]:
q1 = User.objects.filter(id__gte=5)
q1

<QuerySet [<User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>]>

In [17]:
q2 = User.objects.filter(id__lte=9)
q2

<QuerySet [<User: yash>, <User: John>, <User: Ricky>, <User: sharukh>, <User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>]>

In [18]:
q1.union(q2)

<QuerySet [<User: yash>, <User: John>, <User: Ricky>, <User: sharukh>, <User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>]>

In [19]:
q2.union(q1)

<QuerySet [<User: yash>, <User: John>, <User: Ricky>, <User: sharukh>, <User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>]>

In [20]:
q3 = EventVillain.objects.all()
q3

<QuerySet []>

In [21]:
q1.union(q3)

OperationalError: SELECTs to the left and right of UNION do not have the same number of result columns

쿼리셋의 필드와 데이터 유형이 서로 다르기 때문에 union 실패.  
공통된 필드만 가져온 뒤 union을 수행할 수 있음

In [22]:
Hero.objects.all().values_list(
    "name", "gender"
).union(
Villain.objects.all().values_list(
    "name", "gender"))

<QuerySet []>

### 6. 필요한 열만 골라 조회하려면 어떻게 하나요?

- 쿼리셋의 `values` 메서드와 `values_list` 메서드
- `only` 메서드

이름이 R로 시작하는 모든 사용자의 이름(first_name)과 성(last_name)을 구해 봅시다.  
데이터베이스 시스템의 부하를 줄이기 위해 그 외의 열은 가져오지 않겠습니다.

In [23]:
'''
- values
'''

qs1 = User.objects.filter(
    first_name__startswith='R'
    ).values('first_name', 'last_name')

In [24]:
str(qs1.query)

'SELECT "auth_user"."first_name", "auth_user"."last_name" FROM "auth_user" WHERE "auth_user"."first_name" LIKE R% ESCAPE \'\\\''

In [25]:
qs2 = User.objects.filter(
    first_name__startswith='R'
    ).only('first_name', 'last_name')
str(qs2.query)

'SELECT "auth_user"."id", "auth_user"."first_name", "auth_user"."last_name" FROM "auth_user" WHERE "auth_user"."first_name" LIKE R% ESCAPE \'\\\''

### 7. 장고에서 서브쿼리 식을 사용할 수 있나요?

장고에서 SQL 서브쿼리(subquery, 질의문 내의 하위 질의) 식을 사용할 수 있습니다.

In [26]:
'''
auth_user 모델과 일 대 일(OneToOne) 관계로 연결된 UserParent 모델이 있다고 합시다.
UserParent 모델에서 auth_user 를 가진 행을 모두 구할 수 있습니다.
'''

from django.db.models import Subquery

users = User.objects.all()
UserParent.objects.filter(user_id__in=Subquery(users.values('id')))

<QuerySet []>

Category 모델의 각 행 별로, 가장 선한 Hero 행을 구하기.

In [27]:
'''
benevolence_factor에 따라 내림차순으로 정렬
category=OuterRef('pk')으로 서브쿼리로 사용할 준비

OuterRef: 서브쿼리의 queryset이 외부 쿼리의 필드를 참조할 경우 사용
참조: https://docs.djangoproject.com/en/3.0/ref/models/expressions/
'''
hero_qs = Hero.objects.filter(
    category=OuterRef('pk')
    ).order_by('-benevolence_factor')

'''
most_benevolent_hero=Subquery()
서브쿼리에 별칭을 붙여 Category 쿼리셋 안에서 사용

hero_qs.values('name')[:1]
서브쿼리에서 첫 번째 행의 name 필드를 구함

annotate: queryset의 각 오브젝트에 쿼리 표현식 주석을 닮
>>> q = Blog.objects.annotate(number_of_entries=Count('entry'))
>>> q[0].number_of_entries
42
참조: https://docs.djangoproject.com/en/3.0/ref/models/querysets/#django.db.models.query.QuerySet.annotate
'''
qs = Category.objects.all().annotate(
      most_benevolent_hero=Subquery(
         hero_qs.values('name')[:1]
        )
    )

In [28]:
'''
SELECT "entities_category"."id",
       "entities_category"."name",

  (SELECT U0."name"
   FROM "entities_hero" U0
   WHERE U0."category_id" = ("entities_category"."id")
   ORDER BY U0."benevolence_factor" DESC
   LIMIT 1) AS "most_benevolent_hero"
FROM "entities_category"

'''
str(qs.query)

'SELECT "entities_category"."id", "entities_category"."name", (SELECT U0."name" FROM "entities_hero" U0 WHERE U0."category_id" = ("entities_category"."id") ORDER BY U0."benevolence_factor" DESC  LIMIT 1) AS "most_benevolent_hero" FROM "entities_category"'

### 8. 필드의 값을 서로 비교하여 항목을 선택할 수 있나요?

이름(first_name) 이 'R' 로 시작하는 User 모델의 행을 구하려면   `User.objects.filter(first_name__startswith='R')`와 같이 코드를 작성
  
필드와 필드를 서로 비교
  
예를 들어, 이름(first_name) 을 성(last_name) 과 비교하여 선택하는 것이죠.  
이럴 때 F 객체를 사용합니다.

In [29]:
# 실습을 위한 샘플 데이터 생성
User.objects.create_user(email="shabda@example.com", username="shabda", first_name="Shabda", last_name="Raaj")
User.objects.create_user(email="guido@example.com", username="Guido", first_name="Guido", last_name="Guido")

<User: Guido>

In [30]:
# 이름과 성이 동일한 사용자
User.objects.filter(last_name=F('first_name'))

<QuerySet [<User: Guido>]>

`F` 객체는 annotate 메서드로 계산해 둔 필드를 가리킬 때도 사용 가능  
예를 들어, 이름의 첫 글자와 성의 첫 글자가 동일한 사용자를 구하고 싶다면 `Substr("first_name", 1, 1)` 를 사용할 수 있음

In [31]:
User.objects.create_user(email="guido@example.com", username="Tim", first_name="Tim", last_name="Teters")

<User: Tim>

In [32]:
'''
class Substr(expression, pos, lengh=None, **extra)
field나 expression에서 pos부터 length까지의 문자열을 리턴
참조: https://docs.djangoproject.com/en/3.0/ref/models/database-functions/
'''
from django.db.models.functions import Substr

User.objects.annotate(first=Substr('first_name', 1, 1), last=Substr('last_name', 1, 1)).filter(first=F('last'))

<QuerySet [<User: Guido>, <User: Tim>]>

`F` 객체에 `__gt`, `__lt` 등의 룩업 적용 가능

### 9. FileField에 파일이 들어있지 않은 행은 어떻게 구할 수 있나요?

```python
no_files_objects = MyModel.objects.filter(
    Q(file='') | Q(file=None)
    )
```

### 10. 두 모델을 결합(JOIN)하려면 어떻게 하나요?

SQL에서는 JOIN 문을 이용해 동일한 값을 가진 열을 기준으로 두 표를 결합할 수 있습니다.

In [33]:
a1 = Article.objects.select_related('reporter')
str(a1.query)

'SELECT "events_article"."id", "events_article"."headline", "events_article"."pub_date", "events_article"."reporter_id", "events_article"."slug", "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "events_article" INNER JOIN "auth_user" ON ("events_article"."reporter_id" = "auth_user"."id") ORDER BY "events_article"."headline" ASC'

In [34]:
a2 = Article.objects.filter(reporter__username='John')
str(a2.query)

'SELECT "events_article"."id", "events_article"."headline", "events_article"."pub_date", "events_article"."reporter_id", "events_article"."slug" FROM "events_article" INNER JOIN "auth_user" ON ("events_article"."reporter_id" = "auth_user"."id") WHERE "auth_user"."username" = John ORDER BY "events_article"."headline" ASC'

### 11. 두번째로 큰 항목은 어떻게 구하죠?

첫번째 항목은 `first()` 메서드로, 마지막 항목은 `last()` 메서드로 구할 수 있습니다.  
하지만 N번째 항목을 구하는 메서드는 제공되지 않습니다.  
그 대신, 파이썬의 인덱싱 연산을 이용할 수 있습니다.

In [35]:
user = User.objects.order_by('-last_login')[1]
user.first_name

'John'

In [36]:
user = User.objects.order_by('-last_login')[2]
user.first_name

'Ricky'

`User.objects.order_by('-last_login')[2]`처럼 쿼리셋에 인덱스 연산을 할때,  
장고 ORM은 데이터베이스에서 전체 데이터를 가져와서 인덱싱하는게 아니라,  
`LIMIT ... OFFSET` SQL 구문을 이용해 필요한 데이터만 읽어 온다.

실제 생성되는 SQL 질의문

```sql
SELECT "auth_user"."id",
       "auth_user"."password",
       "auth_user"."last_login",
       "auth_user"."is_superuser",
       "auth_user"."username",
       "auth_user"."first_name",
       "auth_user"."last_name",
       "auth_user"."email",
       "auth_user"."is_staff",
       "auth_user"."is_active",
       "auth_user"."date_joined"
FROM "auth_user"
ORDER BY "auth_user"."last_login" DESC
LIMIT 1
OFFSET 2
```

### 12. 특정 열의 값이 동일한 항목은 어떻게 찾나요?

`first_name` 이 서로 동일한 사용자들을 구한다고 합시다. 특정 열에서 중복된 값을 찾을 때는 아래와 같이 `Count` 를 구한 뒤 중복 수를 기준으로 골라내면 됩니다.

In [37]:
# 샘플 데이터 추가
data = [
    ['johny', 'john', 'Smith', 'john@example.com'],
    ['paul', 'Paul', 'Jones', 'paul@example.com'],
    ['johny1', 'John', 'Smith', 'johny@example.com']
]
for i in data:
    User.objects.create(
        username=i[0],
        first_name=i[1],
        last_name=i[2],
        email=i[3]
    )

In [38]:
duplicates = User.objects.values(
    'first_name'
    ).annotate(name_count=Count('first_name')).filter(name_count__gt=1)
duplicates

<QuerySet [{'first_name': 'John', 'name_count': 2}]>

In [39]:
str(duplicates.query)

'SELECT "auth_user"."first_name", COUNT("auth_user"."first_name") AS "name_count" FROM "auth_user" GROUP BY "auth_user"."first_name" HAVING COUNT("auth_user"."first_name") > 1'

In [40]:
# 중복 항목 구하기
records = User.objects.filter(first_name__in=[item['first_name'] for item in duplicates])
print([item.id for item in records])

[2, 16]


In [41]:
str(records.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE "auth_user"."first_name" IN (John)'

### 13. 쿼리셋에서 고유한 필드 값을 가진 항목은 어떻게 구하나요?

In [42]:
distinct = User.objects.values(
            'first_name'
        ).annotate(
            name_count=Count('first_name')
        ).filter(name_count=1)
records = User.objects.filter(first_name__in=[item['first_name'] for item in distinct])

In [43]:
for i in records:
    print(i.username)

yash
Ricky
sharukh
Ritesh
Billy
Radha
sohan
Raghu
rishab
shabda
Guido
Tim
johny
paul


In [44]:
str(distinct.query)

'SELECT "auth_user"."first_name", COUNT("auth_user"."first_name") AS "name_count" FROM "auth_user" GROUP BY "auth_user"."first_name" HAVING COUNT("auth_user"."first_name") = 1'

In [45]:
str(records.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE "auth_user"."first_name" IN (Billy, Guido, Paul, Radha, Raghu, Ricky, Rishabh, Ritesh, Shabda, Sharukh, Sohan, Tim, Yash, john)'

### 14. Q 객체를 이용해 복잡한 질의를 수행하는 방법은 무엇인가요?

`Q` 객체를 이용하면 SQL 질의문의 WHERE 절에 해당하는 기능을 온전히 활용할 수 있습니다.

In [46]:
# OR
queryset = User.objects.filter(
               Q(first_name__startswith='R') | Q(last_name__startswith='D'))
queryset

<QuerySet [<User: Ricky>, <User: Ritesh>, <User: Radha>, <User: Raghu>, <User: rishab>]>

In [47]:
str(queryset.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE \'\\\' OR "auth_user"."last_name" LIKE D% ESCAPE \'\\\')'

In [48]:
# AND
queryset = User.objects.filter(
               Q(first_name__startswith='R') & Q(last_name__startswith='D'))
queryset

<QuerySet [<User: Ricky>, <User: Ritesh>, <User: rishab>]>

In [49]:
str(queryset.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE \'\\\' AND "auth_user"."last_name" LIKE D% ESCAPE \'\\\')'

In [50]:
'''
이름(first_name)이 ‘R’로 시작하되,
성(last_name)에 ‘Z’가 포함되지 않은 사용자를 모두 구하기.
'''
queryset = User.objects.filter(
           Q(first_name__startswith='R')& ~Q(last_name__startswith='Z'))

str(queryset.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE ("auth_user"."first_name" LIKE R% ESCAPE \'\\\' AND NOT ("auth_user"."last_name" LIKE Z% ESCAPE \'\\\'))'

생성된 SQL 질의문

```sql
SELECT "auth_user"."id",
       "auth_user"."password",
       "auth_user"."last_login",
       "auth_user"."is_superuser",
       "auth_user"."username",
       "auth_user"."first_name",
       "auth_user"."last_name",
       "auth_user"."email",
       "auth_user"."is_staff",
       "auth_user"."is_active",
       "auth_user"."date_joined"
FROM "auth_user"
WHERE ("auth_user"."first_name"::text LIKE R%
       AND NOT ("auth_user"."last_name"::text LIKE Z%))
```

### 15. 기록된 항목의 집계를 구할 수 있나요?

장고 ORM에는 SQL의 일반적인 집계 기능을 수행하는 `Max`, `Min`, `Avg`, `Sum` 등의 함수가 있습니다.

In [51]:
from django.db.models import Avg, Max, Min, Sum, Count

User.objects.all().aggregate(Avg('id'))

{'id__avg': 8.5}

In [52]:
User.objects.all().aggregate(Max('id'))

{'id__max': 16}

In [53]:
User.objects.all().aggregate(Min('id'))

{'id__min': 1}

In [54]:
User.objects.all().aggregate(Sum('id'))

{'id__sum': 136}

### 16. 항목을 무작위로 뽑고 싶습니다. 효율적인 방법이 있을까요?

Category Model
```python
class Category(models.Model):
    name = models.CharField(max_length=100)

    class Meta:
        verbose_name_plural = "Categories"

    def __str__(self):
        return self.name
```

In [55]:
# 1. order_by 메서드로 '무작위'로 지정
def get_random():
    return Category.objects.order_by('?').first()
# 실행 비용이 비싸고, 성능이 느릴 수 있음

In [56]:
import random

for i in range(1000):
    Category.objects.create(name='%032x' % random.getrandbits(128))

In [57]:
len(Category.objects.all())

1000

In [58]:
# 2. 정렬대신 마지막 ID를 이용
# ID의 최대값을 구하고, 1과 마지막 ID 사이에 난수를 생성하여, ID가 이 난수와 동일한 항목을 구함
# 삭제 등 ID가 중간에 비어있는 경우 사용할 수 없음

from django.db.models import Max
from entities.models import Category

def get_random2():
    max_id = Category.objects.all().aggregate(max_id=Max('id'))['max_id']
    while True:
        pk = random.randint(1, max_id)
        category = Category.objects.filter(pk=pk).first()
        if category:
            return category

In [59]:
%time
get_random()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


<Category: 4771ae1c1f8d41bf0037bac67c1e76e1>

In [60]:
%time
get_random2()

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs


<Category: d717c46daca4b500070f649feb2c3d21>

### 17. 장고가 지원하지 않는 데이터베이스 함수를 사용할 수 있나요?

장고에는 `Lower`, `coalesce`, `Max` 등의 데이터베이스 함수가 포함되어 있음.  
하지만 특정 데이터베이스 시스템의 전용 함수들은 제공되지 않음.  

이런 경우 **`Func`** 객체를 사용하면 됨.

In [71]:
# PostgreSQL에는 fuzzystrmatch 확장 기능이 있음
# 텍스트 데이터의 유사도를 측정하기 위한 함수들이 있음
data = ["Zeus", "ZeuX", "Xeus", "Poseidon"]

Origin.objects.create(name='tester')

for i in data:
    Hero.objects.create(
        name=i,
        category = Category.objects.get(id=1),
        origin = Origin.objects.get(id=1),
        gender = "Male",
        description = "A greek God",
        benevolence_factor=80,
        arbitrariness_factor=1
    )

In [177]:
from django.db.models import Func, F
from Levenshtein import distance

Hero.objects.annotate(
    link_zeus=Func(
        F('name'),
        function='distance',
        template="%(function)s(%(expressions)s, 'Zeus')"
    )
)

OperationalError: no such function: distance

In [12]:
from django.db.models import Func, F
from Levenshtein import distance

class LevenshteinLikeZeus(Func):
    distance(, 'Zeus')

TypeError: distance expected two Strings or two Unicodes

In [9]:
a = LevenshteinLikeZeus(F('name'))

In [11]:
Hero.objects.annotate(like_zeus=LevenshteinLikeZeus(F('name')))

OperationalError: no such function: None

In [188]:
Hero.objects.annotate(
    like_zeus=LevenshteinLikeZeus(F("name"))
    ).filter(
        like_zeus__lt=2
    )

OperationalError: no such function: distance

`pip install python-Levenshtein`

In [209]:
a.template

'%(function)s(%(expressions)s)'

In [13]:
class LevenshteinLikeZeus(Func):
    function='levenshtein'
    template="%(function)s(%(expressions)s, 'Zeus')"

In [14]:
a = LevenshteinLikeZeus()

In [15]:
a.template

"%(function)s(%(expressions)s, 'Zeus')"

In [16]:
a.function

'levenshtein'

In [21]:
a = Func(F('name'), function=distance, template="%(function)s(expressions)s, 'Zeus'")

In [25]:
class test(Func):
    function=distance
    template=f"{function}({expressions}, 'Zeus')"

NameError: name 'expressions' is not defined

In [23]:
type(a)

django.db.models.expressions.Func

In [24]:
Hero.objects.annotate(
    like_zeus=a)

OperationalError: near "<": syntax error

..... 안되네... 시간을 너무 잡아먹어서 나중에 다시.

## 항목을 생성·갱신·삭제하는 방법


### 1. 여러 개의 행을 한번에 생성하는 방법이 있나요?

`bulk_create` 메서드를 이용하면 여러 신규 객체를 한번에 저장할 수 있다.  
해당 동작은 질의를 여러번 수행하지 않는다.

In [62]:
Category.objects.all().count()

1000

In [63]:
Category.objects.bulk_create(
    [Category(name="God"),
     Category(name="Demi God"),
     Category(name="Mortal")]
)

[<Category: God>, <Category: Demi God>, <Category: Mortal>]

In [64]:
Category.objects.all().count()

1003

### 2. 기존에 저장된 행을 복사해 새로 저장하는 방법은 무엇인가요?

모델 인스턴스를 저장할 때, `pk`필드 값이 `None`으로 지정되어 있으면 데이터베이스에 새 행으로 저장된다. 그 외의 모든 필드 값은 그대로 복제된다.

In [72]:
Hero.objects.all().count()

4

In [73]:
hero = Hero.objects.first()

In [74]:
hero.pk = None

In [75]:
hero.save()
Hero.objects.all().count()

5

### 3. 특정 모델의 항목이 하나만 생성되도록 강제하는 방법이 있나요?

특정 모델의 항목이 단 하나만 생성되도록 강제하고 싶을 때가 있습니다.  
프로그램의 환경 설정 기록, 공유 자원에 대한 잠금 제어 등을 예로 들 수 있습니다.

In [ ]:
class Origin(models.Model):
    name = models.CharField(max_length=100)
    
    def save(self, *args, **kwargs):
        if self.__class__.objects.count():
            self.pk = self.__class__.objects.first().pk
        super().save(*args, **kwargs)

`save()`를 재정의하여 `pk` 필드를 이미 존재하는 값으로 지정하도록 강제함.  
이로써 이미 존재할 때 `create()`를 호출하는 경우 `IntegerityError`가 발생하도록 함

### 4. 모델 인스턴스를 저장할 때, 다른 모델에 반정규화된 필드를 함께 갱신하는 방법이 있나요?

```python
class Category(models.Model):
    name = models.CharField(max_length=100)
    hero_count = models.PositiveIntegerField()
    villain_count = models.PositiveIntegerField()

    class Meta:
        verbose_name_plural = "Categories"


class Hero(models.Model):
    name = models.CharField(max_length=100)
    category = models.ForeignKey(Category, on_delete=models.CASCADE)
    # ...


class Villain(models.Model):
    name = models.CharField(max_length=100)
    category = models.ForeignKey(Category, on_delete=models.CASCADE)
    # ...
```

Hero 모델과 Villain모델의 항목을 새로 저장할 때, Category 모델의 `hero_count`와 `villain_count`를 갱신하도록 한다.

```python
# 1. update()으로 데이터베이스의 갱신을 수행하도록 한다.

class Hero(models.Model):
    # ...

    def save(self, *args, **kwargs):
        if not self.pk:
            Category.objects.filter(pk=self.category_id).update(hero_count=F('hero_count')+1)
        super().save(*args, **kwargs)


class Villain(models.Model):
    # ...

    def save(self, *args, **kwargs):
        if not self.pk:
            Category.objects.filter(pk=self.category_id).update(villain_count=F('villain_count')+1)
        super().save(*args, **kwargs)
```

```python
# 2. 시그널을 활용한다.
from django.db.models.signals import pre_save
from django.dispatch import receiver

@receiver(pre_save, sender=Hero, dispatch_uid="update_hero_count")
def update_hero_count(sender, **kwargs):
    hero = kwargs['instance']
    if hero.pk:
        Category.objects.filter(pk=hero.category_id).update(hero_count=F('hero_count')+1)
        
@receiver(pre_save, sender=Villain, dispatch_uid="update_villain_count")
def update_villain_count(sender, **kwargs):
    villain = kwargs['instance']
    if villain.pk:
        Category.objects.filter(pk=villain.category_id).update(villain_count(F('villain_count')+1)
```

- 반정규화 필드에 영향을 끼치는 모델을 통제할 수 있다면 `save()`를 재정의
- 반정규화 필으에 영향을 끼치는 모델을 통제할 수 없다면 시그널을 이용

### 5. TRUNCATE 문을 수행하는 방법이 있나요?

- TRUNCATE: 표에 저장된 모든 항목을 제거하는 명령

장고는 TRUNCATE 문을 실행하는 명령을 제공하지 않지만, `delete()`를 이용해 비슷한 결과를 얻을 수 있음

In [77]:
Category.objects.all().count()

1003

In [78]:
Category.objects.all().delete()

(1008,
 {'events.Epic_participating_heroes': 0,
  'events.EventHero': 0,
  'entities.HeroAcquaintance_friends': 0,
  'entities.HeroAcquaintance_detractors': 0,
  'entities.Hero': 5,
  'entities.Category': 1003})

In [79]:
Category.objects.all().count()

0

- 위 코드는 잘 동작하지만 TRUNCATE 문이 아니라 `DELETE FROM...` 과 같은 SQL 질의를 수행
- 삭제 항목이 많으면 처리 속도가 느림
- `trungate` 명령이 필요하다면 `classmethod`로 추가하면 됨

```python
class Category(models.Model):
    ...
    @classmethod
    def truncate(cls):
        with connection.cursor() as cursor:
            corsor.execute('TRUNCATE TABLE "{0}" CASCADE'.format(cls._meta.db_table))
```

### 6. 모델 인스턴스가 생성·갱신될 때 발생하는 시그널에는 어떤 것이 있나요?

- `pre_init`
- `post_init`
- `pre_save`
- `post_save`
- `pre_delete`
- `post_delete`

시그널을 이용하면 `save()`를 재정의하는 것과 비슷한 효과를 가짐.

- 다른 사람(외부 라이브러리 등)이 당신의 앱의 `save()`를 커스터마이즈하도록 허용하려면 직접 시그널을 발생시켜야 함
- 통제할 수 없는 앱의 `save()`가 호출될 때 원하는 코드가 실행되도록 하려면 `post_save`나 `pre_save` 시그널을 이용해야 함
- 통제할 수 있는 앱의 저장 방식을 손 볼 때는 `save()`를 재정의해야 함


### 7. 시간 정보를 다른 양식으로 변환하여 데이터베이스에 저장하려면 어떻게 해야 하나요?

장고의 `dateparser` 모듈이나 파이썬 표준 라이브러리를 이용해서 변환

In [80]:
user = User.objects.get(id=1)

In [81]:
date_str = "2020-01-06"

In [82]:
# 1.
from django.utils.dateparse import parse_date
temp_data = parse_date(date_str)
a1 = Article(headline="String converted to date", pub_date=temp_data, reporter=user)
a1.save()
a1.pub_date

datetime.date(2020, 1, 6)

In [83]:
# 2.
from datetime import datetime
temp_date = datetime.strptime(date_str, "%Y-%m-%d").date()
a2 = Article(headline="String converted to date way 2", pub_date=temp_date, reporter=user)
a2.save()
a2.pub_date

datetime.date(2020, 1, 6)

## 조회 결과를 정렬하는 방법

### 1. 쿼리셋을 오름차순/내림차순으로 정렬할 수 있나요?

`order_by()`으로 쿼리셋을 정렬

In [84]:
q1 = User.objects.all().order_by('date_joined') # 오름차순
str(q1.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" ORDER BY "auth_user"."date_joined" ASC'

In [85]:
q2 = User.objects.all().order_by('-date_joined') # 내림차순
str(q2.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" ORDER BY "auth_user"."date_joined" DESC'

In [86]:
# 정렬 기준 필드를 여러개 지정
q3 = User.objects.all().order_by('date_joined', '-last_login')
str(q3.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" ORDER BY "auth_user"."date_joined" ASC, "auth_user"."last_login" DESC'

```sql
SELECT "auth_user"."id",
       -- More fields
       "auth_user"."date_joined"
FROM "auth_user"
ORDER BY "auth_user"."date_joined" ASC,
         "auth_user"."last_login" DESC
```

### 2. 대문자·소문자를 구별하지 않고 정렬하려면 어떻게 하나요?

`order_by()`으로 쿼리셋을 정렬할때, 대문자가 소문자보다 높은 우선순위가 부여됨

In [90]:
User.objects.all().order_by('username').values_list('username', flat=True)


<QuerySet ['Billy', 'Guido', 'John', 'Radha', 'Raghu', 'Ricky', 'Ritesh', 'Tim', 'johny', 'johny1', 'paul', 'rishab', 'shabda', 'sharukh', 'sohan', 'yash']>

In [91]:
# 대소문자 구별하지 않으려면 'Lower'를 사용
from django.db.models.functions import Lower

User.objects.all().order_by(Lower('username')).values_list('username', flat=True)

<QuerySet ['Billy', 'Guido', 'John', 'johny', 'johny1', 'paul', 'Radha', 'Raghu', 'Ricky', 'rishab', 'Ritesh', 'shabda', 'sharukh', 'sohan', 'Tim', 'yash']>

In [92]:
# 'annotate()'로 'Lower'를 적용한 열을 준비하고, 그 열을 기준으로 정렬해도 됨
User.objects.annotate(
    lower_name=Lower('username')
    ).order_by('lower_name').values_list('username', flat=True)

<QuerySet ['Billy', 'Guido', 'John', 'johny', 'johny1', 'paul', 'Radha', 'Raghu', 'Ricky', 'rishab', 'Ritesh', 'shabda', 'sharukh', 'sohan', 'Tim', 'yash']>

### 3. 여러 개의 필드를 기준으로 정렬하는 방법이 있나요?

`order_by()`에 여러 개의 인자를 전달하면 됨

In [96]:
from django.contrib.auth.models import User

qs = User.objects.all().order_by('is_active', '-last_login', 'first_name')
qs

<QuerySet [<User: Billy>, <User: Guido>, <User: John>, <User: johny1>, <User: paul>, <User: Radha>, <User: Raghu>, <User: Ricky>, <User: rishab>, <User: Ritesh>, <User: shabda>, <User: sharukh>, <User: sohan>, <User: Tim>, <User: yash>, <User: johny>]>

In [95]:
str(qs.query)

'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" ORDER BY "auth_user"."is_active" ASC, "auth_user"."last_login" DESC, "auth_user"."first_name" ASC'

### 4. 외래 키로 연결된 다른 표의 열을 기준으로 정렬할 수 있나요?

```python
class Category(models.Model):
    name = models.CharField(max_length=100)


class Hero(models.Model):
    # ...
    name = models.CharField(max_length=100)
    category = models.ForeignKey(Category, on_delete=models.CASCADE)
```

Category와 Hero 모델은 외래키로 연결되어 있음.  

In [97]:
'''
Hero 모델의 쿼리셋을 category 필드 순으로 정렬하되,
category가 같은 항목은(Hero의) name 필드 순으로 정렬
'''

qs = Hero.objects.all().order_by(
    'category__name', 'name'    # '__' 으로 연결된 모델의 필드를 가리킬 수 있음
)

str(qs.query)

'SELECT "entities_hero"."id", "entities_hero"."name", "entities_hero"."alternative_name", "entities_hero"."category_id", "entities_hero"."origin_id", "entities_hero"."gender", "entities_hero"."description", "entities_hero"."added_by_id", "entities_hero"."added_on", "entities_hero"."is_immortal", "entities_hero"."benevolence_factor", "entities_hero"."arbitrariness_factor", "entities_hero"."headshot", "entities_hero"."father_id", "entities_hero"."mother_id", "entities_hero"."spouse_id" FROM "entities_hero" INNER JOIN "entities_category" ON ("entities_hero"."category_id" = "entities_category"."id") ORDER BY "entities_category"."name" ASC, "entities_hero"."name" ASC'

```sql
SELECT "entities_hero"."id",
       "entities_hero"."name",
       -- more fields
FROM "entities_hero"
INNER JOIN "entities_category" ON ("entities_hero"."category_id" = "entities_category"."id")
ORDER BY "entities_category"."name" ASC,
         "entities_hero"."name" ASC
```

### 5. 계산 필드를 기준으로 정렬할 수 있나요?

```python
class Category(models.Model):
    name = models.CharField(max_length=100)


class Hero(models.Model):
    # ...
    name = models.CharField(max_length=100)
    category = models.ForeignKey(Category, on_delete=models.CASCADE)
```

Category 항목들을 각 Category 항목에 속한 Hero 항목의 개수에 따라 정렬하려면,  
`annotate()`로 계산 필드를 준비하여 기준을 삼으면 된다.

In [100]:
qs = Category.objects.annotate(
    hero_count=Count('hero')
).order_by(
    '-hero_count'
)
str(qs.query)

'SELECT "entities_category"."id", "entities_category"."name", COUNT("entities_hero"."id") AS "hero_count" FROM "entities_category" LEFT OUTER JOIN "entities_hero" ON ("entities_category"."id" = "entities_hero"."category_id") GROUP BY "entities_category"."id", "entities_category"."name" ORDER BY "hero_count" DESC'